In [1]:
import pandas as pd
from dspy.datasets import DataLoader
file_path = "../.data/all_raretrible.txt"
data = open(file_path, "r", encoding="utf-8").read().split("\n")
data = [x.strip().split(",") for x in data if len(x) > 0]
data = [[x[0], ",".join(x[1:])] for x in data]

columns=["label", "text",]
# 转换 data 到 pandas DataFrame
df = pd.DataFrame(data, columns=columns)

# 创建 dspy.datasets 对象
dl = DataLoader()
# fields 选择指定列 input_keys
all_dataset = dl.from_pandas(
    df,
    fields=columns,
    input_keys=("text",),
)
from collections import defaultdict
import random


def stratified_sample(from_dataset, label_attr, *dataset_sizes):
    label_groups = defaultdict(list)
    for item in from_dataset:
        label = getattr(item, label_attr)
        label_groups[label].append(item)

    result_sets = [[] for _ in dataset_sizes]

    for label, items in label_groups.items():
        random.shuffle(items)
        label_sizes = [size // len(label_groups) for size in dataset_sizes]
        start = 0
        for i, size in enumerate(label_sizes):
            end = start + size
            result_sets[i].extend(items[start:end])
            start = end

    all_items = [item for items in label_groups.values() for item in items]
    for i, (result_set, target_size) in enumerate(zip(result_sets, dataset_sizes)):
        shortage = target_size - len(result_set)
        if shortage > 0:
            result_sets[i].extend(random.sample(all_items, shortage))

    return result_sets
train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
from collections import Counter

train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
for dataset in [train_set, test_set, sample_set]:
    label_counts = Counter(item.get("label") for item in dataset)
    print(label_counts)

Counter({'感激': 6, '高兴': 4, '中性': 2, '焦急': 2, '惊讶': 2, '抱怨': 2, '生气': 2})
Counter({'中性': 8, '感激': 7, '焦急': 7, '惊讶': 7, '高兴': 7, '抱怨': 7, '生气': 7})
Counter({'感激': 15, '生气': 15, '中性': 14, '焦急': 14, '惊讶': 14, '高兴': 14, '抱怨': 14})


In [4]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)

# trans = transfer_lang.TransferLang.from_dataset(test_set,lang="马来语")
# trans.modelname = 'glm-4-air'
# ret = trans.do()
# print(ret.content)
# trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set,random=True,modelname = 'glm-4-air')
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

Example({'label': '感激', 'text': '设置了,好的,谢谢。晚一些再看看吧'}) (input_keys={'text'})
Example({'label': '感激', 'text': '谢谢您‘’'}) (input_keys={'text'})
Example({'label': '感激', 'text': '好的,谢谢[愉快][愉快]'}) (input_keys={'text'})
Example({'label': '感激', 'text': '那行,谢谢'}) (input_keys={'text'})


Example({'label': '感激', 'text': '好的哈,谢谢了哦'}) (input_keys={'text'})
Example({'label': '感激', 'text': '嗯,好。谢谢'}) (input_keys={'text'})
Example({'label': '感激', 'text': '嗯嗯,那就好,没其他问题了,谢谢'}) (input_keys={'text'})
Example({'label': '中性', 'text': '您好,这两天开了中央空调致暖,结果中央空调室外主机下面有水流出,请问什么原因,可以帮忙解决吗?谢谢'}) (input_keys={'text'})
Example({'label': '中性', 'text': '我吃饭前借的,吃完饭就还了'}) (input_keys={'text'})
Example({'label': '中性', 'text': '什么是那个码?'}) (input_keys={'text'})
Example({'label': '中性', 'text': '两天之内可以到吗'}) (input_keys={'text'})
Example({'label': '中性', 'text': '提前还款是打电话还款吗'}) (input_keys={'text'})
Example({'label': '中性', 'text': '你们下款最慢多久'}) (input_keys={'text'})
Example({'label': '中性', 'text': '我的菜还没收到'}) (input_keys={'text'})
Example({'label': '焦急', 'text': '亲,快些哈'}) (input_keys={'text'})
Example({'label': '焦急', 'text': '刚刷一笔怎么还没到账'}) (input_keys={'text'})
Example({'label': '焦急', 'text': '麻烦尽量快点哦!加急一下吧!!!'}) (input_keys={'text'})
Example({'label': '焦急', 'text': '能快点不,亲'}) (input_keys={'text'})
Exa

In [6]:
# for item in test_set:
#     print(item)
ana.show_error()

0.5
Example({'label': '高兴', 'text': '么么哒?', 'pred_label': '中性', 'main': '中性', 'score': '0.60'}) (input_keys={'text'})
Example({'label': '中性', 'text': '您好,这两天开了中央空调致暖,结果中央空调室外主机下面有水流出,请问什么原因,可以帮忙解决吗?谢谢', 'pred_label': '焦急', 'main': '中性', 'score': '0.70'}) (input_keys={'text'})
Example({'label': '焦急', 'text': '都是现货的东西几天不发货,过年了谁还等他啊', 'pred_label': '抱怨', 'main': '负面', 'score': '0.85'}) (input_keys={'text'})
Example({'label': '高兴', 'text': '我爱你燕子', 'pred_label': '中性', 'main': '中性', 'score': '0.70'}) (input_keys={'text'})
Example({'label': '中性', 'text': '我的菜还没收到', 'pred_label': '焦急', 'main': '中性', 'score': '0.80'}) (input_keys={'text'})
Example({'label': '高兴', 'text': '么么哒?', 'pred_label': '中性', 'main': '中性', 'score': '0.60'}) (input_keys={'text'})
Example({'label': '中性', 'text': '你们下款最慢多久', 'pred_label': '焦急', 'main': '中性', 'score': '0.75'}) (input_keys={'text'})
Example({'label': '感激', 'text': '嗯嗯,那就好,没其他问题了,谢谢', 'pred_label': '中性', 'main': '中性', 'score': '0.70'}) (input_keys={'text'})
Ex